### QA notebook - VAC object selection
#### Andressa Wille
#### 19/02/22

Plots: Distribuições de magnitudes; Distribuições de magnitude errors; Distribuições de cores; Distribuições de photo-z; Distribuições de photo-z error; Number counts normalizados com referências da literatura (vide product log); Plot magnitude X error; Plot magnitude X redshift; Color-magnitude diagrams; Color-color diagrams.

Dados: VAC 6850


### Imports

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pylab as pl
from astropy.table import Table
from statistics import mode
from matplotlib.colors import LogNorm
import sys
from funcs1 import *

### VAC

In [ ]:
vac = get_vac(pid='6850',
            vac_schema='cluster', 
            bands=['g', 'r', 'i', 'z', 'y'],
            sample_frac = 0.1)

In [ ]:
vac.describe()

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']
colors = ['g-r', 'r-i', 'i-z', 'z-y']

### Distribuição de magnitudes

In [ ]:
for i, band in enumerate(bands):
    plt.figure()
    plt.hist(vac[f'mag_{band}'], histtype='bar', bins='auto', ec="cornflowerblue")
    plt.xlabel("mag "+band)
    plt.ylabel("Counts")
    plt.xlim(10, 40)
    plt.yscale('log')
    plt.grid(True)
plt.tight_layout()

### Distribuição de erros nas magnitudes

In [ ]:
#logspace
bins = np.linspace(0,40,200)
for i, band in enumerate(bands):
    plt.figure()
    plt.hist(vac[f'magerr_{band}'], bins=bins, ec="cornflowerblue", fc="cornflowerblue")
    plt.xlabel("mag err "+band)
    plt.ylabel("Counts")
    plt.yscale('log')
    plt.grid(True)
pl.tight_layout()

### Distribuição de cores

In [ ]:
for i, band in enumerate(colors):
    plt.figure() 
    vac[colors[i]] = vac[f'mag_{bands[i]}'] - vac[f'mag_{bands[i+1]}']
    plt.hist(vac[colors[i]], bins='auto', ec="cornflowerblue")
    plt.xlabel(band)
    plt.ylabel("Counts")
    plt.yscale('log')
    plt.grid(True)
plt.tight_layout()

### Distribuição de redshift

In [ ]:
plt.hist(vac['z_best'], bins='auto', ec="cornflowerblue", fc="cornflowerblue")
plt.xlabel("photo-$z$")
plt.ylabel("Counts")
plt.title("Redshift Distribution")
plt.tight_layout()

### Distribuição de erro no redshift

In [ ]:
plt.hist(vac['err_z'], bins='auto', ec="cornflowerblue", fc="cornflowerblue")
plt.xlabel("err photo-$z$")
plt.ylabel("Counts")
plt.xlim(0, 2)
plt.title("Redshift Error Distribution")
plt.tight_layout()

### Number counts

In [ ]:
area = 29.05
bins = np.linspace(15,25,21)
for i, band in enumerate(bands):
    plt.figure()
    n, bins, patches = plt.hist(vac[f'mag_{band}'], histtype='step', weights=(vac[f'mag_{band}']/area),
             bins=bins, color='black')
    #print(np.argmax(n))
    #print(bins)
    maximo = (bins[np.argmax(n)] + bins[np.argmax(n)+1])/2
    plt.vlines(maximo, 10e-1, 10e6, linestyles='dotted', label=maximo)
    plt.xlabel("mag "+band)
    plt.ylabel("$N_{gal} deg^{-2}0.5mag^{-1}$")
    plt.xlim(15, 25)
    plt.ylim(10e-1, 10e5)
    plt.yscale('log')
    plt.grid(True)
    plt.legend()
plt.tight_layout()

### Magnitude x erro

In [ ]:
for i, band in enumerate(bands):
    #pl.figure()
    pl.figure(figsize=[10,4])
    pl.subplot(121) 
    mag = np.array(vac[f'mag_{band}'])
    err = np.array(vac[f'magerr_{band}'])
    mask = (mag>14)&(mag<32)&(err<10)
    #MASCARA?
    pl.hexbin(mag[mask], err[mask],None,  mincnt=1, cmap='viridis', gridsize=[200,100], bins='log')
    cbar = pl.colorbar()
    pl.xlabel("mag "+band)
    pl.ylabel("err")
    pl.grid(True)
pl.tight_layout()

### Magnitude x redshift

In [ ]:
for i, band in enumerate(bands):
    pl.figure(figsize=[10,4])
    pl.subplot(121) 
    mag = np.array(vac[f'mag_{band}'])
    z = np.array(vac['z_best'])
    mask = (mag>14)&(mag<32)&(z<=2)
    pl.hexbin(mag[mask], z[mask], None,  mincnt=1, cmap='viridis', gridsize=[200,100], bins='log')
    cbar = pl.colorbar()
    pl.xlabel("mag "+band)
    pl.ylabel("redshift")
    pl.grid(True)
pl.tight_layout()

### Diagrama cor-magnitude

In [ ]:
zbin_center, istar = np.loadtxt('istar.asc', unpack=True)
istar_dic = {}
for i in range(len(istar)):
    istar_dic[zbin_center[i]] = istar[i]

In [ ]:
vacs = [vac]
titles = ["VAC 6850"]

In [ ]:
x, y = 'mag_i', 'r-i'
z_low, z_up = 0.5, 0.6
color_cut = 0.65
x_range, y_range = [16, 24.5], [-1., 2]

plot_loop(vacs, x, y, z_low, z_up, color_cut, x_range, y_range, titles, istar_dic=istar_dic)

### Diagrama cor-cor

In [ ]:
CC(vac['mag_g'], vac['mag_r'], vac['mag_i'], vac['mag_z'], 'g', 'r', 'i', 'z', 24., 'log')